In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [13]:
df = pd.read_table('azh.txt')
df['tf'] = df["CDR3.amino.acid.sequence"].apply(lambda x: not ("*" in x or "~" in x))
df = df[df['tf'] == True]
df.tail()

,Umi.count,Umi.proportion,Read.count,Read.proportion,CDR3.nucleotide.sequence,CDR3.amino.acid.sequence,V.gene,J.gene,D.gene,V.end,J.start,D5.end,D3.end,VD.insertions,DJ.insertions,Total.insertions,tf
346164,0,0.0,1,4.329235e-07,TGTGCAGCAAAGATCTTT,CAAKIF,TRAV29/DV5,TRAJ30,NaN,9,12,-1,-1,-1,-1,2,True
346165,0,0.0,1,4.329235e-07,TGCCTGAGGATCATGTTT,CLRIMF,TRAV4,TRAJ31,NaN,4,10,-1,-1,-1,-1,5,True
346166,0,0.0,1,4.329235e-07,TGCCTGAAATTAACCTTT,CLKLTF,TRAV4,TRAJ48,NaN,4,5,-1,-1,-1,-1,0,True
346167,0,0.0,1,4.329235e-07,TGCGGGGAGTTCTATTTT,CGEFYF,TRAV30,TRAJ49,NaN,5,7,-1,-1,-1,-1,1,True
346168,0,0.0,1,4.329235e-07,TGCATCGTCAGAGTCTTT,CIVRVF,TRAV26-1,TRAJ26,NaN,13,14,-1,-1,-1,-1,0,True


In [3]:
max(df['CDR3.amino.acid.sequence'].apply(len))

22

In [4]:
chars = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'X', 'Y']
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

maxlen = 22

In [5]:
def vectorization(data):
    prots = data['CDR3.amino.acid.sequence']
    X = np.zeros((len(prots), maxlen, len(chars)), dtype=np.bool)
    for i, sequence in enumerate(prots):
        for t, char in enumerate(sequence):
            X[i, t, char_indices[char]] = 1
    return np.fliplr(X)

In [19]:
X_base = vectorization(df[df['Umi.count'] > 4])
y_base = - np.log(np.array(df[df['Umi.count'] > 4]['Umi.proportion']))
sample = df[df['Umi.count'] <= 4]
X_little = vectorization(sample[sample['Umi.count'] > 0])
y_little = - np.log(np.array(sample[sample['Umi.count'] > 0]['Umi.proportion']))
X_zero = vectorization(df[df['Umi.count'] == 0])
y_zero = np.zeros(44349)

In [20]:
def generator(X_1, X_2, X_0,y_1,y_2,y_0):
    while 1:
        for i in range(10):
            yield (np.concatenate([X_1[i*len(X_1)/10:(i+1)*len(X_1)/10], X_2[i*500:(i+1)*500], X_0[i*100:(i+1)*100]]), np.concatenate([y_1[i*len(y_1)/10:(i+1)*len(y_1)/10], y_2[i*500:(i+1)*500], y_0[i*100:(i+1)*100]]))

In [61]:
np.concatenate([X_base[5:25],X_little[3:13], X_zero[4:87]]).shape

X_base.shape

(11699, 22, 21)

In [8]:
from keras.layers import Dense, LSTM, Merge, Activation, BatchNormalization
from keras.models import Sequential
from keras.optimizers import RMSprop

Using Theano backend.
Using gpu device 1: Tesla K20Xm (CNMeM is disabled, cuDNN 5105)
/home/cerebro/anaconda3/lib/python3.5/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [9]:
model = Sequential()
    # Dropouts + BN works quite well. Without dropouts the learning process is faster
    # but I'm quite sure that this is due to the overtraining.
model.add(LSTM(128, dropout_W = .2, dropout_U = .2, input_shape=(maxlen, len(chars))))
model.add(BatchNormalization())
    # Don't use L2 regularization ActivityRegularizer(l2 = .3) on the output Dense layer - 
    # it pushes the output to too strict boundaries, so the output will be always in [0,1]
model.add(Dense(1))
    # Log-transorm the data and multiple by "-1" so it's possible to use ReLU instead of linear activations.
model.add(Activation('relu'))

    # Very small learning rate because with the higher rate nans start to occur.
optimizer = RMSprop(lr=0.000003)
    # Use MSLE instead of MSE because in case of log transformation of our data
    # the model will pay too much attention to optimization of differences for small
    # receptors, which has large negative (= large absolute value) values.
model.compile(loss='mse', optimizer=optimizer)


In [22]:
model.fit_generator(generator(X_base, X_little, X_zero,y_base,y_little,y_zero), 7080, 10)

/home/cerebro/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:4: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


Epoch 1/10
8849/7080 [=====================================] - 1s - loss: 102.3029

/home/cerebro/anaconda3/lib/python3.5/site-packages/keras/engine/training.py:1470: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '



Epoch 2/10
7080/7080 [==============================] - 0s - loss: 114.8603     
Epoch 3/10
8849/7080 [=====================================] - 1s - loss: 103.6812
Epoch 4/10
7080/7080 [==============================] - 0s - loss: 113.3363     
Epoch 5/10
8849/7080 [=====================================] - 1s - loss: 105.3337
Epoch 6/10
7080/7080 [==============================] - 0s - loss: 111.6118     
Epoch 7/10
8849/7080 [=====================================] - 1s - loss: 107.4360
Epoch 8/10
7080/7080 [==============================] - 0s - loss: 109.0167     
Epoch 9/10
7080/7080 [==============================] - 0s - loss: 115.9136     
Epoch 10/10
8849/7080 [=====================================] - 1s - loss: 102.1504
